# Using rail_fsps to generate galaxy rest-frame spectral energy distributions and  compute apparent magnitudes

author: Luca Tortorelli, Eric Charles
    
last run successfully: February 17th, 2023
    
This notebook demonstrates how to use rail_fsps to generate galaxy rest-frame spectral energy distributions (SEDs) with FSPS and how to compute apparent magnitudes from them. We will also show how to use Ceci to do the same.

In [ ]:
import rail
from rail.creation.galaxy_modelling.fsps_sed_modeler import *
from rail.creation.galaxy_modelling.fsps_photometry_creator import *
from rail.core.stage import RailStage
import os
import numpy as np
import matplotlib.pyplot as plt

We'll start by setting up the Rail data store. RAIL uses ceci, which is designed for pipelines rather than interactive notebooks, the data store will work around that and enable us to use data interactively. See the rail/examples/goldenspike/goldenspike.ipynb example notebook for more details on the Data Store.

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

We generate some mock input data for the sed modeler class that we store in .npy array files.

In [ ]:
DRN = '/src/rail/examples/testdata'
metallicities = np.array([0.0, -0.1])
np.save(os.path.join(DRN, 'rail_fsps_test_metallicities.npy'), metallicities)
ages = np.array([1.,7.])
np.save(os.path.join(DRN, 'rail_fsps_test_ages.npy'), ages)
veldisp = np.array([150.,200.])
np.save(os.path.join(DRN, 'rail_fsps_test_veldisp.npy'), veldisp)
gasioniz = np.array([-1.,-2.])
np.save(os.path.join(DRN, 'rail_fsps_test_gasioniz.npy'),gasioniz)
gasmetal = np.array([0.0, -0.1])
np.save(os.path.join(DRN, 'rail_fsps_test_gasmetal.npy'),gasmetal)
dust1 = np.array([0.2, 0.3])
np.save(os.path.join(DRN, 'rail_fsps_test_dust1.npy'),dust1)
dust2 = np.array([0.2, 0.3])
np.save(os.path.join(DRN, 'rail_fsps_test_dust2.npy'),dust2)
dustindex = np.array([-0.1, 0.1])
np.save(os.path.join(DRN, 'rail_fsps_test_dustindex.npy'),dustindex)
dustgamma = np.array([0.01, 0.07])
np.save(os.path.join(DRN, 'rail_fsps_test_dustgamma.npy'),dustgamma)
dustumin = np.array([1., 5.])
np.save(os.path.join(DRN, 'rail_fsps_test_dustumin.npy'),dustumin)
dustqpah = np.array([1., 3.])
np.save(os.path.join(DRN, 'rail_fsps_test_dustqpah.npy'),dustqpah)
agnbol = np.array([10**(-3), 10**(-2)])
np.save(os.path.join(DRN, 'rail_fsps_test_agnbol.npy'),agnbol)
agntorus = np.array([10, 20])
np.save(os.path.join(DRN, 'rail_fsps_test_agntorus.npy'),agntorus)

Let's create an FSPSSedModeler class object. The latter has a number of configuration parameters to set. All the parameters have default values. Therefore, if the user is not sure about a particular value for a certain parameter, the latter can be left at default. The parameter names are the same as the ones used in the Python bindings of FSPS. A short description of each parameter can be found in src/rail/creation/galaxy_modelling/fsps_sed_modeler.py. 

The parameters that do not have correspondance with FSPS are:
- min_wavelength: minimum rest-frame wavelength to consider when generating the rest-frame SED
- max_wavelength: maximum rest-frame wavelength to consider when generating the rest-frame SED
- galaxy_metallicities: path to the npy file containing the metallicity values of the rest-frame SEDs in units of $\log_{10}(Z / Z_{\odot})$. If the parameter sfh_type=3 and zcontinuous=3, a tabulated star-formation history file containing values of the galaxy metallicity as function of time must be provided. In this case, the metallicities values in the npy file are ignored.
- galaxy_ages: path to the npy file containing the values of the age of the Universe at the galaxy redshift in Gyr, this represents the maximum age of the oldest stellar population. If redshifts are set to 0, then it's the age at which the rest-frame SED is evaluated. 
- galaxy_vel_disp: path to the npy file containing the velocity dispersion values in km/s or Anstrom (smooth_velocity=True/False) with which FSPS smooths the spectrum in the wavelength range (min_wavelength, max_wavelength). Default to None to apply default value in FSPS.
- gas_ionization_params: path to npy file containing the the values of the logarithm of the gas ionization parameter. Default to None to apply default value in FSPS.
- gas_metallicity_params: path to npy file containing the values of the logarithm of the gas-phase metallicity. Default to None to apply default value in FSPS.
- dust_birth_cloud_params: path to npy file containing the dust parameters describing the attenuation of young stellar light. Default to None to apply default value in FSPS.
- dust_diffuse_params: path to npy file containing the dust parameters describing the attenuation of old stellar light. Default to None to apply default value in FSPS.
- dust_powerlaw_modifier_params: path to npy file containing the dust parameters describing the power-law modifier to the shape of the Calzetti et al. 2000 attenuation curve. Default to None to apply default value in FSPS.
- dust_emission_gamma_params: path to npy file containing relative contributions of dust heated at Umin (parameter of the Draine and Li 2007 dust emission model). Default to None to apply default value in FSPS.
- dust_emission_umin_params: path to npy file containing the values of the minimum radiation field strenght Umin (parameter of the Draine and Li 2007 dust emission model). Default to None to apply default value in FSPS.
- dust_emission_qpah_params: path to npy file containing the grain size distribution in mass in PAHs (parameter of the Draine and Li 2007 dust emission model). Default to None to apply default value in FSPS.
- fraction_agn_bol_lum_params: path to npy file containing the fractional contribution of the AGN component luminosity with respect to the stellar bolometric luminosity. Default to None to apply default value in FSPS.
- agn_torus_opt_depth_params: path to npy file containing the optical depths of the AGN dust torus. Default to None to apply default value in FSPS.
- physical_units: False (True) to output rest-frame spectra in units of $L_{\odot} \mathrm{Hz}^{-1}$ ( $erg\ s^{-1} \mathrm{Hz}^{-1}$).

We run the FSPSSedModeler in sequential mode. Note that each galaxy spectrum takes a few seconds to generate, so it is advisable to proceed in this way only for a limited sample of objects or for testing the code.

In [ ]:
sedmodel = FSPSSedModeler.make_stage(name='FSPS_sed_model', zcontinuous=1, add_neb_emission=True,
                                     add_neb_continuum=True, nebemlineinspec=True,
                                     smooth_velocity=True, smooth_lsf=False, imf_type=1,
                                     min_wavelength=1000, max_wavelength=10000, sfh_type=3,
                                     dust_type=4, galaxy_metallicities=os.path.join(DRN, 'rail_fsps_test_metallicities.npy'),
                                     galaxy_ages=os.path.join(DRN, 'rail_fsps_test_ages.npy'),
                                     galaxy_vel_disp=os.path.join(DRN, 'rail_fsps_test_veldisp.npy'),
                                     gas_ionization_params=os.path.join(DRN, 'rail_fsps_test_gasioniz.npy'),
                                     gas_metallicity_params=os.path.join(DRN, 'rail_fsps_test_gasmetal.npy'),
                                     dust_birth_cloud_params=os.path.join(DRN, 'rail_fsps_test_dust1.npy'), 
                                     dust_diffuse_params=os.path.join(DRN, 'rail_fsps_test_dust2.npy'),
                                     dust_powerlaw_modifier_params=os.path.join(DRN, 'rail_fsps_test_dustindex.npy'),
                                     dust_emission_gamma_params=os.path.join(DRN, 'rail_fsps_test_dustgamma.npy'),
                                     dust_emission_umin_params=os.path.join(DRN, 'rail_fsps_test_dustumin.npy'),
                                     dust_emission_qpah_params=os.path.join(DRN, 'rail_fsps_test_dustqpah.npy'),
                                     fraction_agn_bol_lum_params=os.path.join(DRN, 'rail_fsps_test_agnbol.npy'),
                                     agn_torus_opt_depth_params=os.path.join(DRN, 'rail_fsps_test_agntorus.npy'),
                                     physical_units=True)

Here we show the example where we provide tabulated star-formation histories to generate the final SED. In this case, FSPS outputs the emission per total stellar mass. We call the fit_model() function to generate the model SEDs.
The fit_model function parameters have the same meaning of the Python-FSPS ones, besides:
- tabulated_sfh_files: list of paths to the txt files containing the tabulated star-formation histories. The files are two-columns text files, with cosmic time in Gyr in the first column and star-formation rate in $M_{\odot}/\mathrm{yr}$ in the second column. If sfh_type=3 and zcontinuous=3, the third column is the metallicity at each age, in units of absolute metallicity (e.g. Z=0.019 for Padova isochrones).
- tabulated_lsf_file: path to the txt file containing the wavelength-dependent Gaussian line spread function that will be applied to the SEDs. The files are two-columns text files, with wavelength in Angstron in the first column and sigma in the second column. Its effect is going to be applied if and only if both smooth_lsf and smooth_velocity are True.

In [ ]:
tabulated_sfh_files = [os.path.join(DRN, 'rail_fsps_test_sfh.txt'), os.path.join(DRN, 'rail_fsps_test_sfh.txt')]
redshifts = np.array([0.9740825916412452, 0.9740825916412452])
np.save(os.path.join(DRN, 'rail_fsps_test_redshifts.npy'),redshifts)
zred = os.path.join(DRN, 'rail_fsps_test_redshifts.npy')
restrame_sed_models = sedmodel.fit_model(compute_vega_mags=False, vactoair_flag=False, add_agb_dust_model=True,
                                         add_dust_emission=True, add_igm_absorption=False, add_stellar_remnants=True,
                                         compute_light_ages=False, cloudy_dust=False, agb_dust=1.0, tpagb_norm_type=2,
                                         dell=0.0, delt=0.0, redgb=1.0, agb=1.0, fcstar=1.0, fbhb=0.0, sbss=0.0, pagb=1.0,
                                         zmet=1, pmetals=2.0, imf_upper_limit=120, imf_lower_limit=0.08,
                                         imf1=1.3, imf2=2.3, imf3=2.3, vdmc=0.08, mdave=0.5, evtype=-1, use_wr_spectra=1,
                                         logt_wmb_hot=0.0, masscut=150.0, igm_factor=1.0, tau=np.ones(1),
                                         const=np.zeros(1), sf_start=np.zeros(1), sf_trunc=np.zeros(1), fburst=np.zeros(1),
                                         tburst=np.ones(1), sf_slope=np.zeros(1), dust_tesc=7.0, dust_clumps=-99.,
                                         frac_nodust=0.0, frac_obrun=0.0, dust_index=-0.7, mwr=3.1, uvb=1.0, wgp1=1,
                                         wgp2=1, wgp3=1, zred=zred, 
                                         tabulated_sfh_files=tabulated_sfh_files, tabulated_lsf_file='')

We plot the first spectrum to check that the SED generation worked correctly.

In [ ]:
plt.clf()
plt.plot(restrame_sed_models.data['wavelength'], restrame_sed_models.data['restframe_seds'][0],
         lw=2, color='black')
plt.xlim(1000, 10000)
plt.xlabel(r'wavelength [$\AA$]')
plt.ylabel(r'luminosity density [$\mathrm{erg \ s^{-1} \ Hz^{-1}}$]')
plt.show()

Now we use these rest-frame SEDs to generate LSST photometry at user-provided redshifts.
For this we need the FSPSPhotometryCreator class. The class parameters are:

- filter_data: path to the npy file containing filter transmission curves. The npy file must be a structured array with key names '{}_filter_wave'.format(waveband) and '{}_filter_trans'.format(waveband). Wavelengths must be in Angstrom.
- rest_frame_sed_models: ModelHandle or pickle file containing the rest-frame SEDs generated by the FSPSSedModeler class.
- galaxy_redshifts: same meaning as above.
- Om0, Ode0, w0, wa, h: cosmological parameters for a w0waCDM cosmology
- use_planck_cosmology: True to overwrite the cosmological parameters with Planck15 cosmology model from Astropy
- physical_units: same meaning as above.

In [ ]:
from rail.core.utils import RAILDIR
default_rail_files_folder = os.path.join(RAILDIR, 'rail', 'examples', 'testdata')

phot_creator = FSPSPhotometryCreator.make_stage(name='FSPS_photometry', 
                                                filter_data=os.path.join(default_rail_files_folder, 'lsst_filters.npy'),
                                                rest_frame_sed_models='model_FSPS_sed_model.pkl',
                                                galaxy_redshifts=os.path.join(DRN, 'rail_fsps_test_redshifts.npy'),
                                                Om0=0.3, Ode0=0.7, w0=-1, wa=0., h=0.7,
                                                use_planck_cosmology=True, physical_units=True)

We call the sample method to generate LSST photometry once the class has been initialised. The output is a table in Fits format, with three columns: sequential ID, redshifts, apparent AB magnitudes

In [ ]:
output_table = phot_creator.sample()

In [ ]:
print(output_table.data)

The RAIL stages can be chained together conveniently using Ceci. The following is an example of how the FSPSSedGenerator and FSPSPhotometryCreator can be run as part of the pipeline Ceci stages. 

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [ ]:
sedmodel_forceci = FSPSSedModeler.make_stage(name='FSPS_sed_model', zcontinuous=1, add_neb_emission=True,
                                             add_neb_continuum=True, nebemlineinspec=True,
                                             smooth_velocity=True, smooth_lsf=False, imf_type=1,
                                             min_wavelength=1000, max_wavelength=10000, sfh_type=3,
                                             dust_type=4, galaxy_metallicities=os.path.join(DRN, 'rail_fsps_test_metallicities.npy'),
                                             galaxy_ages=os.path.join(DRN, 'rail_fsps_test_ages.npy'),
                                             galaxy_vel_disp=os.path.join(DRN, 'rail_fsps_test_veldisp.npy'),
                                             gas_ionization_params=os.path.join(DRN, 'rail_fsps_test_gasioniz.npy'),
                                             gas_metallicity_params=os.path.join(DRN, 'rail_fsps_test_gasmetal.npy'),
                                             dust_birth_cloud_params=os.path.join(DRN, 'rail_fsps_test_dust1.npy'), 
                                             dust_diffuse_params=os.path.join(DRN, 'rail_fsps_test_dust2.npy'),
                                             dust_powerlaw_modifier_params=os.path.join(DRN, 'rail_fsps_test_dustindex.npy'),
                                             dust_emission_gamma_params=os.path.join(DRN, 'rail_fsps_test_dustgamma.npy'),
                                             dust_emission_umin_params=os.path.join(DRN, 'rail_fsps_test_dustumin.npy'),
                                             dust_emission_qpah_params=os.path.join(DRN, 'rail_fsps_test_dustqpah.npy'),
                                             fraction_agn_bol_lum_params=os.path.join(DRN, 'rail_fsps_test_agnbol.npy'),
                                             agn_torus_opt_depth_params=os.path.join(DRN, 'rail_fsps_test_agntorus.npy'),
                                             physical_units=True)

In [ ]:
phot_creator_forceci = FSPSPhotometryCreator.make_stage(name='FSPS_photometry', 
                                                        filter_data=os.path.join(default_rail_files_folder, 'lsst_filters.npy'),
                                                        rest_frame_sed_models='model_FSPS_sed_model.pkl',
                                                        galaxy_redshifts=os.path.join(DRN, 'rail_fsps_test_redshifts.npy'),
                                                        Om0=0.3, Ode0=0.7, w0=-1, wa=0., h=0.7,
                                                        use_planck_cosmology=True, physical_units=True)

In [ ]:
pipe = ceci.Pipeline.interactive()
stages = [sedmodel_forceci, phot_creator_forceci]
for stage in stages:
    pipe.add_stage(stage)

In [ ]:
pipe.initialize(dict(input='model_FSPS_sed_model.pkl'), dict(output_dir='./temp_output', log_dir='./logs', resume=False, nprocess=2), 
                './temp_output/pipe_sed_fsps_config.yml')

#pipe.initialize(dict(stages_config='./temp_output/pipe_sed_fsps_config.yml'), dict(input=''), dict(output_dir='./temp_output', log_dir='./logs', resume=False, nprocess=2))
pipe.save('./temp_output/pipe_saved.yml')
pr = ceci.Pipeline.read('./temp_output/pipe_saved.yml')
pr.run()


The creation of galaxy SEDs is a computationally intensive process. To speed things up, it is convenient to parallelize the process using MPI. To do that, one needs to set the parameters into the configuration file pipe_saved_config.yml. An example command to be run in the command line with n_cores is:

mpiexec -n n_cores --mpi python3 -m rail SedGenerator --input=test_fsps_sed.fits --name=sed_generator_test --config=pipe_saved_config.yml --output=output_sed_generator_test.fits
